# Coursework 2 - Mathematics for Machine Learning

## Due: 12/12/2025

## CID: 02232170

***
***

## Exercise 1: Approximation theory

### Exercise 1.1

#### Question 1:

$$ f(x) = A_2\sigma(A_1x + b_1)$$
Noting some important details: $x$ is of size d and non-negative ($ \in [0,1]^d$) and the width of each layer is required to be $d+3$.  We want $w$ ReLU layers which is the same number of rows as $A_1$ and columns as $A_2$. We will rename A1 to $A^{(1)}$  A2 to $A^{(2)}$ and b1 as $b^{(1)}$ to denote the layer they belong to and reference their rows and columns easily.
$$
   [\sigma(A^{(1)}x + b^{(1)})]_i = \sigma(\sum_{j = 1}^d A_{i, j}^{(1)}x + b_i^{(1)}) \tag{ ith element after activation}
$$
We can rewrite f as: $\sum_{i = 1}^w A_i^{(2)}[\sigma(A^{(1)}x + b^{(1)})]_i$.

<br> The new network we will construct will separate each of the rows of $A^{(1)}$ into w hidden layers, and use $ReLU$ as the activation function. what we need to do <br>
1. Preserve x across all layers(of width d) so that we can apply each row of $A^{(1)}$ independently <br>
2. compute $sigma(A^{(1)}x + b^{(1)})$ across w hidden and ReLU layers (of width 1) <br>
3. store the sum of positive and negative values from previous layers (of width 1 each). We do this because $ReLU$ will be applied on our new layers and only outputs non-negative values, so we need to keep track of positive and negative contributions separately. Note that $x \in [0,1]^d$ so we dont need to worry about $ReLU$ erasing any values from x. <br>
4. The final layer will subtract the negative sum from the positive sum. <br>
Dimensionally, we have d+3 nodes per layer, which meets the requirement. <br>

Let's define the layers: <br>
- $W^{(1)}$ : First layer weights of size (d+3) x d 
$$
   W^{(1)} = \begin{bmatrix}
   I \\
   A_1^{(1)} \\
    \bold{0} \\
    \bold{0} 
   \end{bmatrix}
$$
Where I is the identity matrix of size d x d,  $\bold{0}$ are zero vectors of size 1 x d, $A_1^{(1)}$ is the first row of $A^{(1)}$. <br> We will denote our new bias vector as $c^{(i)}$ for layer i. 
$$
c^{(1)} = \begin{bmatrix}
   \bold{0} \\
   b_1^{(1)} \\
    0 \\
    0 
\end{bmatrix}
$$
Thus post-activation we have the following calculations:
$$
\sigma(W^{(1)}x + c^{(1)}) = \begin{bmatrix}
   x \\
   \sigma(A_1^{(1)}x + b_1^{(1)}) \\
    0 \\
    0
\end{bmatrix}
$$
- For layer 2 to w we get the following input, Weight and bias: <br>
$$

    W^{(i)} = \begin{bmatrix}
    I & 0 & 0 & 0\\
    A_i^{(1)} & 0 & 0 & 0 \\
    \bold{0} & \sigma(A_{i-1}^{(2)}) & 1 & 0 \\
    \bold{0} & \sigma(-A_{i-1}^{(2)}) & 0 & 1
    \end{bmatrix}
$$
$$
x' = \begin{bmatrix}
    x \\
    \sigma(A_{i-1}^{(1)}x + b_{i-1}^{(1)}) \\
    \text{Sum of positive contributions from previous layers} \\
    \text{Sum of negative contributions from previous layers}
    \end{bmatrix}
    \;
c^{(i)} = \begin{bmatrix}
    \bold{0} \\
    b_i^{(1)} \\
    0 \\
    0 
    \end{bmatrix}
$$ 
Note that in all our $c^{(i)}$ vectors, $\bold{0}$ is a zero vector of size d x 1. <br>
Explanation of the weight matrix: <br>
- First d rows preserve x  due to Identity matrix<br>
- (d+1)th row computes $A_i^{(1)}x + b_i^{(1)}$ <br>
- (d+2)th row adds the positive contribution from this layer(if any) to the sum of positive contributions from previous layers <br>
- (d+3)th row adds the negative contribution from this layer(if any) to the sum of negative contributions from previous layers <br> 
I have used $\sigma(A_{i-1}^{(2)})$ to denote whether the output from the previous layer was positive or negative, which determines whether we add to the positive or negative sum. But, if we were to construct it in reality, we would put a zero on row d+2 if $A_{i-1}^{(2)}$ is negative itself otherwise. Likewise for row d+3. This makes it so that the negative and positive contributions are stored as positive values and 0 is added to the layers. We have $I_2$ in the bottom right of W^(i) so that we sum the previously gained contributions to the new contribution. <br><br>

To make it more clear we will write the output of 1 layer depending on the sign of $A_{i-1}^{(2)}$: <br>

If $A_{i-1}^{(2)}$ is positive: <br>
$$
    \implies \begin{bmatrix}
    x \\
    \sigma(A_{i-1}^{(1)}x + b_{i-1}^{(1)}) \\
    \text{Sum of positive contributions from previous layers} + \sigma(A_{i-1}^{(1)}x + b_{i-1}^{(1)}) \\
    \text{Sum of negative contributions from previous layers}
    \end{bmatrix}
$$
If $A_{i-1}^{(2)}$ is negative: <br>
$$
    \implies \begin{bmatrix}
    x \\
    \sigma(A_{i-1}^{(1)}x + b_{i-1}^{(1)}) \\
    \text{Sum of positive contributions from previous layers} \\
    \text{Sum of negative contributions from previous layers} + \sigma(A_{i-1}^{(1)}x + b_{i-1}^{(1)})
    \end{bmatrix}
$$

- Finally for the output layer we have: <br>
$$
    W^{output} = \begin{bmatrix} 0_{1 \times d} & A^{(2)}_w & -1 & 1 \end{bmatrix}
$$
such that when applying to the output of the wth hidden layer we get the calculation:
$$
\begin{bmatrix} 0_{1 \times d} & A^{(2)}_w & -1 & 1 \end{bmatrix} \times
\begin{bmatrix} x \\ \sigma(A_{w}^{(1)}x + b_{w}^{(1)}) \\ 
\sum_{j=1}^{w-1} \sigma(A_{j}^{(1)}x + b_{j}^{(1)})\mathbb{1}_{A_{j}^{(2)}>0} \\
\sum_{j=1}^{w-1} \sigma(A_{j}^{(1)}x + b_{j}^{(1)})\mathbb{1}_{A_{j}^{(2)}<0}
\end{bmatrix} \\


 = A^{(2)}_w \sigma(A_{w}^{(1)}x + b_{w}^{(1)}) + 
 \sum_{j=1}^{w-1} \sigma(A_{j}^{(1)}x + b_{j}^{(1)})\mathbb{1}_{A_{j}^{(2)}>0} 
 - \sum_{j=1}^{w-1} \sigma(A_{j}^{(1)}x + b_{j}^{(1)})\mathbb{1}_{A_{j}^{(2)}<0} 
 = A^{(2)}_w \sigma(A_{w}^{(1)}x + b_{w}^{(1)}) + 
 \sum_{j=1}^{w-1} A_{j}^{(2)} \sigma(A_{j}^{(1)}x + b_{j}^{(1)})  \\
 = \sum_{i = 1}^w A_i^{(2)}[\sigma(A^{(1)}x + b^{(1)})]_i = f(x)
$$
Therefore, we define $N(x)$ as the following:
$$
N(x) = W^{output} \sigma(W^{(w)} \sigma(W^{(w-1)} ... \sigma(W^{(1)}x + c^{(1)}) ... + c^{(w-1)}) + c^{(w)}) 
$$ 
and have proved that $$ N(x) = f(x) \; \forall x \in [0,1]^d$$


#### Question 2:

Enter your answer here
In our previous answer, we assumed that $ x \in [0,1]^d$. However, if we relax this assumption and allow $x$ to be in $[-1,1]^d$. One way to handle this is to introduce d extra rows in W and c, $W \in R^{(2d + 3)\times d}$ and $c \in R^{2d + 3}$. We will use it to store $ -x$ alongside x in the first layer.
Our new Matrices and vectors will look like this:
$$
    W^{(1)} = \begin{bmatrix}
    I \\
    -I \\
    A_1^{(1)} \\
     \bold{0} \\
     \bold{0} 
    \end{bmatrix}
$$
where, I is the identity matrix of size d x d,  $\bold{0}$ are zero vectors of size 1 x d, $A_1^{(1)}$ is the first row of $A^{(1)}$. <br> We will denote our new bias vector as $c^{(i)}$ for layer i.
$$
c^{(1)} = \begin{bmatrix}
    \bold{0} \\
    \bold{0} \\
    b_1^{(1)} \\
     0 \\
     0 
\end{bmatrix}
$$
Where $\bold{0}$ is a zero vector of size d x 1. <br>
For $i >1$ we have:
$$
    W^{(i)} = \begin{bmatrix}
    I & \bold{0} & 0 & 0 & 0\\
    \bold{0} & -I & 0 & 0 & 0\\
    A_i^{(1)} & -A_i^{(1)} & 0 & 0 & 0 \\
    \bold{0} & \bold{0} & \sigma(A_{i-1}^{(2)}) & 1 & 0 \\
    \bold{0} & \bold{0} & \sigma(-A_{i-1}^{(2)}) & 0 & 1
    \end{bmatrix}
$$

Finally for the output layer we have: <br>
$$ 
    W^{output} = \begin{bmatrix} 0_{1 \times d} & 0_{1 \times d} & A^{(2)}_w & 1 & -1 \end{bmatrix}
$$
Example Output of hidden layer:
$$
\begin{bmatrix}
    \sigma(x) \\
    \sigma(-x) \\
    \sigma(A_{i-1}^{(1)}x + b_{i-1}^{(1)}) \\
    \sum \sigma(A^{(1)}_j x + b_j^{(1)}) * 1_{A_j^{(2)}>0} \\
    \sum \sigma(A^{(1)}_j x + b_j^{(1)}) * 1_{A_j^{(2)}<0}
\end{bmatrix}
$$

***
### Exercise 1.2

#### Question 1:

We can begin by using the hint.
$$
h(x) = \lim_{\delta \to 0} \frac{f((w+\delta)x) - f(wx)}{\delta}
$$
Let's define $g_\delta(x) = \frac{f((w+\delta)x) - f(wx)}{\delta}$ which is $\in \mathcal{F}_{\sigma, 1} $ since $\mathcal{F}_{\sigma, 1}$ is closed under addition and scalar multiplication from the question.
Using the Taylor expansion of $f$ about $wx$ we have:
$$
f((w+\delta)x) = f(wx) + \delta x f'(wx) + \frac{\delta^2 x^2}{2} f''(z)
$$
for some $z$ between $wx$ and $(w+\delta)x$. Thus we have:
$$
g_\delta(x) = \frac{f((w+\delta)x) - f(wx)}{\delta} \\ =  \frac{f(wx) + \delta x f'(wx) + \frac{\delta^2 x^2}{2} f''(z) - f(wx)}{\delta} \\ =  x f'(wx) + \frac{\delta x^2}{2} f''(z) = h(sx) + \frac{\delta x^2}{2} f''(z)
$$

So,
$$
|| h - g_\delta ||_u = \sup_{x \in [0,1]} |h(x) - g_\delta(x)| \\
= \sup_{x \in [0,1]} | \frac{\delta x^2}{2} f''(z) | \leq \frac{\delta}{2} \sup_{x \in [0,1]} |z^2 f''(z)| \\
\leq \frac{\delta}{2} \sup_{x \in [0,1]} | f''(z)|
$$
I used that $x \in [0,1]$. we know that $z

#### Question 2:

Enter your answer here

#### Question 3:

Enter your answer here

#### Question 4:

Enter your answer here

#### Question 5:

Enter your answer here

#### Question 6:

Enter your answer here

*** 
***

## Exercise 2: Implementation

### Part 1: Image classification


### Question 1
Enter your answer here

### Question 2

### Question 3

### Question 4

### Question 5

### Question 6

***

### Part 2: Fine tuning

### Question 1

### Question 2

### Question 3

### Question 4

### Question 5

***
### Part 3: Adversarial attacks

### Question 1

### Question 2

### Question 3

### Question 4

### Question 5